# 787. Cheapest Flights Within K Stops

There are n cities connected by some number of flights. You are given an array flights where flights[i] = [fromi, toi, pricei] indicates that there is a flight from city fromi to city toi with cost pricei.

You are also given three integers src, dst, and k, return the cheapest price from src to dst with at most k stops. If there is no such route, return -1.

## Example 1:

**Input:** n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 1
**Output:** 200
**Explanation:** 
The graph looks like this:
```
   0
 /   \
100   500
/       \
1 ---100--- 2
```
The cheapest price from city 0 to city 2 with at most 1 stop costs 200, as marked red in the picture.

## Example 2:

**Input:** n = 3, flights = [[0,1,100],[1,2,100],[0,2,500]], src = 0, dst = 2, k = 0
**Output:** 500
**Explanation:** 
The graph looks like this:
```
   0
 /   \
100   500
/       \
1 ---100--- 2
```
The cheapest price from city 0 to city 2 with at most 0 stop costs 500, as marked red in the picture.

## Constraints:

- 1 <= n <= 100
- 0 <= flights.length <= n * (n - 1) / 2
- flights[i].length == 3
- 0 <= fromi, toi < n
- fromi != toi
- 1 <= pricei <= 10^4
- There will not be any multiple flights between two cities.
- 0 <= src, dst, k < n
- src != dst

In [ ]:
from collections import List, defaultdict
from functools import cache

# Bellman-Ford BFS approach
class Solution:
    def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
        
        # Initialize distances array with infinity
        prices = [float('inf')] * n
        prices[src] = 0
        
        # Make k+1 iterations (allows for k stops)
        for i in range(k + 1):
            # Create a copy of current prices
            temp_prices = prices.copy()
            
            for flight in flights:
                from_city, to_city, price = flight
                
                # If the source city is unreachable, skip
                if prices[from_city] == float('inf'):
                    continue
                    
                # If we can reach to_city with a cheaper price through from_city
                if prices[from_city] + price < temp_prices[to_city]:
                    temp_prices[to_city] = prices[from_city] + price
                    
            prices = temp_prices
            
        return prices[dst] if prices[dst] != float('inf') else -1

# DFS with memoization
class Solution2:
    def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
    
        # If we expect k to be small, it's reasonable to assume DFS can still be efficient here
        flightDirectory = defaultdict(dict)
        for (source, dest, price) in flights:
            flightDirectory[source][dest] = price
        
        memo = {}

        # DFS recursive version
        def dfs(k, src):
            if k < 0:
                return -1

            if src == dst:
                return 0

            # Check if we've computed this state before
            if (src, k) in memo:
                return memo[(src, k)]

            minCost = float('inf')
            for nextCity, cost in flightDirectory[src].items():
                remainingCost = dfs(k-1, nextCity)
                if remainingCost != -1:
                    minCost = min(minCost, remainingCost + cost)
            
            memo[(src, k)] = minCost if minCost != float('inf') else -1
            return memo[(src, k)]
            
        return dfs(k+1, src)
    
# DFS with @cache decorator
class Solution3:
    def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
    
        # If we expect k to be small, DFS can still be efficient here w/ memoization
        flightDirectory = defaultdict(dict)
        for (source, dest, price) in flights:
            flightDirectory[source][dest] = price

        # DFS recursive func
        @cache
        def dfs(k, src):
            if k < 0: # no stops left
                return -1

            if src == dst: # found our destination
                return 0

            # Find minCost to dst through any of our adj
            minCost = float('inf')
            for nextCity, cost in flightDirectory[src].items():
                remainingCost = dfs(k-1, nextCity)
                if remainingCost != -1:
                    minCost = min(minCost, remainingCost + cost)
            
            return minCost if minCost != float('inf') else -1
            
        return dfs(k+1, src)


In [ ]:
# TEST CASES

# Solution 3 test
sol3 = Solution3()

test1 = sol3.findCheapestPrice(
    4, [[0,1,100],[1,2,100],[2,0,100],[1,3,600],[2,3,200]], 0, 3, 1
)

test2 = sol3.findCheapestPrice(
    3, [[0,1,100],[1,2,100],[0,2,500]], 0, 2, 1
)

test3 = sol3.findCheapestPrice(
    3, [[0,1,100],[1,2,100],[0,2,500]], 0, 2, 0
)


In [3]:
# SOLUTION WITH STRING INPUT

from collections import defaultdict, deque
from functools import cache
import heapq

# Bellman-Ford BFS approach
class Solution():

    def __init__(self, flights: str):
        self.flightDirectory = self.getFlightDirectory(flights)

    def getFlightDirectory(self, flights: str) -> dict[dict]:
        # "UK:US:FedEx:4,UK:FR:Jet1:2,US:UK:RyanAir:8,CA:UK:CanadaAir:8"
        flightDirectory = defaultdict(lambda: defaultdict(list))
        flightList = flights.split(',')
        for flight in flightList:
            src, dst, airline, cost = flight.split(':')
            flightDirectory[src][dst].append({
                'airline': airline,
                'cost': float(cost)
            })
        return flightDirectory
    
    def findCheapestFlight(self, src: str, dst: str) -> str:
        minCost = float('inf')
        minCostFlight = None
        for flight in self.flightDirectory[src][dst]:
            if flight['cost'] < minCost:
                # minCostFlight = f"{src}:{dst}:{flight['airline']}:{flight['cost']}"
                minCostFlight = flight
        return minCostFlight

    def findRoute(self, src: str, dst: str) -> str:
        # Use BFS to minimize the number of stops
        queue = deque([(src, 0)])
        visited = set([src])
        parent = {src: None}

        while queue:
            v, cost = queue.popleft()
            visited.add(v)

            if v == dst:
                # return "US:UK:FR", 10
                curr, path = dst, []
                while curr:
                    path.append(curr)
                    curr = parent[curr]
                path = path[::-1]
                return (':'.join(path), cost)
            
            for adj in self.flightDirectory[v]:
                flight = self.findCheapestFlight(v, adj)
                if adj not in visited:
                    queue.append((adj, cost + flight['cost']))
                    parent[adj] = v
        
        return None

    def findCheapestRoute(self, src: str, dst: str) -> str:
        # Dijsktra, stop when found
        pq = [(0, src)]
        prev = {src: None}
        visited = set([src])

        while pq:
            (cost, x) = heapq.heappop(pq)

            # process node
            
            if x == dst: # Found destination
                # return "US:UK:FR", 10
                curr, path = dst, []
                while curr:
                    path.append(curr)
                    curr = prev[curr]
                path = path[::-1]
                return (':'.join(path), cost)
            
            # add to visited
            visited.add((x))
            
            for adj, flights in self.flightDirectory[x].items():
                if adj not in visited:
                    for flight in flights:
                        heapq.heappush(pq, (cost + flight['cost'], adj))
                        prev[adj] = x
        
        return None
    
    def findCheapestRouteKStops(self, src: int, dst: int, k: int) -> int:
        
        # If we expect k to be small, DFS can still be efficient here w/ memoization

        # DFS recursive func
        @cache
        def dfs(k, src):
            if k < 0: # no stops left
                return -1

            if src == dst: # found our destination
                return 0

            # Find minCost to dst through any of our adj
            minCost = float('inf')
            for nextCity, flights in self.flightDirectory[src].items():
                for flight in flights:
                    remainingCost = dfs(k-1, nextCity)
                    if remainingCost != -1:
                        minCost = min(minCost, remainingCost + flight['cost'])
            
            return minCost if minCost != float('inf') else -1
            
        return dfs(k+1, src)

In [5]:
# Test string solution

sol = Solution("UK:US:FedEx:4,UK:FR:Jet1:20,US:UK:RyanAir:8,CA:UK:CanadaAir:8,US:FR:AirFrance:10")

test1 = sol.findCheapestRouteKStops('UK', 'FR', 1)
assert(test1 == 14.0)

test2 = sol.findCheapestRouteKStops('UK', 'FR', 0)
assert(test2 == 20.0)